In [10]:

from keras.applications.xception import Xception
from keras.preprocessing import image
import numpy as np
from keras.layers import Dense, Dropout
from keras.models import Model
import os
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
from keras.metrics import categorical_accuracy
from keras.callbacks import Callback, ModelCheckpoint,LearningRateScheduler,EarlyStopping,ReduceLROnPlateau
from keras.optimizers import SGD

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
path1='/content/drive/MyDrive/srtp-xsh/RGBimg/raw_image/s1'
s1_weight='/content/drive/MyDrive/srtp-xsh/RGBimg/raw_image/s1/s1.h5'

In [13]:
train1=path1+'/'+os.listdir(path1)[0]
test1=path1+'/'+os.listdir(path1)[1]
X_1_train=[]
Y_1_train=[]
for x in os.listdir(train1):

  f=train1+'/'+x
  img=image.load_img(f)
  img=image.img_to_array(img)
  img=image.smart_resize(img, (100,200), interpolation='bilinear')
  label=x[3]
  X_1_train.append(img)
  Y_1_train.append(label)

X_1_test=[]
Y_1_test=[]
for x in os.listdir(test1):
  f=test1+'/'+x
  img=image.load_img(f)
  img=image.img_to_array(img)
  img=image.smart_resize(img, (100,200), interpolation='bilinear')
  label=x[3]
  X_1_test.append(img)
  Y_1_test.append(label)



In [14]:
X_1_train=np.asarray(X_1_train,np.float32)
X_1_test=np.asarray(X_1_test,np.float32)
Y_1_train=np.asarray(Y_1_train,np.float32)
Y_1_test=np.asarray(Y_1_test,np.float32)

num_example=X_1_train.shape[0]
arr=np.arange(num_example)
np.random.shuffle(arr)
X_1_train=X_1_train[arr]
Y_1_train=Y_1_train[arr]

if Y_1_train.shape!=(364,52):
  Y_1_train = to_categorical(Y_1_train,num_classes=52)
  Y_1_test = to_categorical(Y_1_test,num_classes=52)

print('shape of training data:',X_1_train.shape)
print('shape of training label:',Y_1_train.shape)
print('shape of test data:',X_1_test.shape)
print('shape of test label:',Y_1_test.shape)

shape of training data: (364, 100, 200, 3)
shape of training label: (364, 52)
shape of test data: (156, 100, 200, 3)
shape of test label: (156, 52)


In [15]:
def minibatches(inputs=None, targets=None, batch_size=None, shuffle=False, aug=None):
    while 1:  
        assert len(inputs) == len(targets)
        if shuffle:
            indices = np.arange(len(inputs))
            np.random.shuffle(indices)
        for start_idx in range( len(inputs) - batch_size ):
            if shuffle:
                excerpt = indices[start_idx:start_idx + batch_size]
                if aug is not None:
                  (inputs[excerpt], targets[excerpt]) = next(aug.flow(inputs[excerpt],targets[excerpt], batch_size=batch_size))
            else:
                excerpt = slice(start_idx, start_idx + batch_size)
                if aug is not None:
                  (inputs[excerpt], targets[excerpt]) = next(aug.flow(inputs[excerpt],targets[excerpt], batch_size=batch_size))
            yield inputs[excerpt], targets[excerpt]



In [16]:
from keras.preprocessing.image import ImageDataGenerator
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")


In [24]:

base_model=Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(100,200,3), pooling='avg')
x=base_model.output
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(52, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [22]:
####随便跑的sgd

In [ ]:
model.compile(optimizer='SGD', loss='categorical_crossentropy',metrics=['categorical_accuracy'])
callbacks = [ModelCheckpoint(s1_weight, monitor='val_loss', save_best_only=True)]
history = model.fit(X_1_train,
                    Y_1_train,
                    epochs=64,
                    validation_split=0.1,
                    callbacks=callbacks)


In [26]:

model.load_weights(s1_weight)
loss, accuracy = model.evaluate(X_1_test,
                    Y_1_test,)
print('loss = {0:.4f}, accuracy = {1:.4f}'.format(loss, accuracy))

5/5 [==============================] - 1s 138ms/step - loss: 0.5077 - categorical_accuracy: 0.8333
loss = 0.5077, accuracy = 0.8333


In [ ]:
####调参部分 （xsh调参自用，糟糕）

In [9]:
# use this for pre-train
for layer in base_model.layers:
    layer.trainable = False

In [10]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 200, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 49, 99, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 49, 99, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 49, 99, 32)   0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='/content/drive/MyDrive/srtp-xsh/RGBimg/raw_image/s1/model.png',show_shapes=True)

In [13]:

his=model.fit_generator(minibatches(X_1_train,Y_1_train,batch_size=6,shuffle=False,aug=None),
                            steps_per_epoch=len(X_1_train)//6,
                            callbacks = [ModelCheckpoint(s1_weight, monitor='val_loss', save_best_only=True, save_weight_only=True)],
                            validation_data=minibatches(X_1_test,Y_1_test,batch_size=6,shuffle=True,aug=None),
                            validation_steps=len(X_1_test)//6,
                            epochs=6)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/6
60/60 [==============================] - 9s 62ms/step - loss: 7.0793 - val_loss: 3.5446


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/6
60/60 [==============================] - 3s 47ms/step - loss: 1.4183 - val_loss: 3.0826
Epoch 3/6
60/60 [==============================] - 3s 46ms/step - loss: 0.9366 - val_loss: 2.3535
Epoch 4/6
60/60 [==============================] - 3s 46ms/step - loss: 0.7330 - val_loss: 2.0755
Epoch 5/6
60/60 [==============================] - 3s 46ms/step - loss: 0.6831 - val_loss: 1.2911
Epoch 6/6
60/60 [==============================] - 3s 47ms/step - loss: 0.8389 - val_loss: 1.7305


In [14]:
# import matplotlib.pyplot as plt
# H=his
# N = 6 # N=epochs 
# plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
# plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
# plt.title("Training Loss on Dataset")
# plt.xlabel("Epoch #")
# plt.ylabel("Loss")
# plt.legend(loc="lower left")
# plt.savefig("plot.png")


In [ ]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

In [16]:
# freeze and train
# 0-35
# (35-55
# 55-75
# 75-95
# 95-115)
# 115-132


In [17]:
for layer in model.layers[0:115]:
   layer.trainable = False
for layer in model.layers[115:133]:
   layer.trainable = True
for layer in model.layers[133:]:
   layer.trainable = False
model.load_weights(s1_weight)
model.compile(optimizer=SGD(lr=0.005, momentum=0.9), loss='categorical_crossentropy',metrics=['categorical_accuracy'])
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
his=model.fit_generator(minibatches(X_1_train,Y_1_train,batch_size=20,shuffle=False,aug=None),
                            steps_per_epoch=len(X_1_train)//20,
                            callbacks = [ModelCheckpoint(s1_weight, monitor='val_loss', save_best_only=True, save_weight_only=True),learning_rate_reduction],
                            validation_data=minibatches(X_1_test,Y_1_test,batch_size=6,shuffle=True,aug=None),
                            validation_steps=len(X_1_test)//20,
                            epochs=20)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
18/18 [==============================] - 7s 167ms/step - loss: 2.6403 - categorical_accuracy: 0.3250 - val_loss: 2.1476 - val_categorical_accuracy: 0.5952


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
18/18 [==============================] - 2s 118ms/step - loss: 2.2247 - categorical_accuracy: 0.2944 - val_loss: 2.2400 - val_categorical_accuracy: 0.4048
Epoch 3/20
18/18 [==============================] - 2s 117ms/step - loss: 2.1507 - categorical_accuracy: 0.2306 - val_loss: 2.5773 - val_categorical_accuracy: 0.1667
Epoch 4/20
18/18 [==============================] - 2s 117ms/step - loss: 1.7937 - categorical_accuracy: 0.4167 - val_loss: 1.7973 - val_categorical_accuracy: 0.2381
Epoch 5/20
18/18 [==============================] - 2s 122ms/step - loss: 1.4493 - categorical_accuracy: 0.5056 - val_loss: 1.4428 - val_categorical_accuracy: 0.4524
Epoch 6/20
18/18 [==============================] - 2s 118ms/step - loss: 1.7448 - categorical_accuracy: 0.3667 - val_loss: 2.1815 - val_categorical_accuracy: 0.4048
Epoch 7/20
18/18 [==============================] - 2s 120ms/step - loss: 1.3811 - categorical_accuracy: 0.5972 - val_loss: 1.8505 - val_categorical_accuracy: 0.2619
Epoc

In [18]:
for layer in model.layers[0:95]:
   layer.trainable = False
for layer in model.layers[95:115]:
   layer.trainable = True
for layer in model.layers[115:]:
   layer.trainable = False
model.load_weights(s1_weight)
model.compile(optimizer=SGD(lr=0.005, momentum=0.9), loss='categorical_crossentropy',metrics=['categorical_accuracy'])
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
his=model.fit_generator(minibatches(X_1_train,Y_1_train,batch_size=20,shuffle=False,aug=None),
                            steps_per_epoch=len(X_1_train)//20,
                            callbacks = [ModelCheckpoint(s1_weight, monitor='val_loss', save_best_only=True, save_weight_only=True),learning_rate_reduction],
                            validation_data=minibatches(X_1_test,Y_1_test,batch_size=6,shuffle=False,aug=None),
                            validation_steps=len(X_1_test)//20,
                            epochs=20)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
18/18 [==============================] - 7s 203ms/step - loss: 2.5897 - categorical_accuracy: 0.2667 - val_loss: 3.1287 - val_categorical_accuracy: 0.1667


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
18/18 [==============================] - 3s 142ms/step - loss: 2.8033 - categorical_accuracy: 0.2083 - val_loss: 3.3179 - val_categorical_accuracy: 0.0238
Epoch 3/20
18/18 [==============================] - 2s 135ms/step - loss: 2.7740 - categorical_accuracy: 0.2306 - val_loss: 2.7383 - val_categorical_accuracy: 0.3333
Epoch 4/20
18/18 [==============================] - 2s 134ms/step - loss: 2.1264 - categorical_accuracy: 0.3833 - val_loss: 3.4429 - val_categorical_accuracy: 0.1667
Epoch 5/20
18/18 [==============================] - 2s 134ms/step - loss: 1.9399 - categorical_accuracy: 0.3611 - val_loss: 2.1475 - val_categorical_accuracy: 0.1429
Epoch 6/20
18/18 [==============================] - 2s 134ms/step - loss: 2.3470 - categorical_accuracy: 0.2028 - val_loss: 2.5495 - val_categorical_accuracy: 0.0000e+00
Epoch 7/20
18/18 [==============================] - 2s 134ms/step - loss: 2.1096 - categorical_accuracy: 0.2861 - val_loss: 2.2399 - val_categorical_accuracy: 0.0000e

In [19]:
# for layer in model.layers[0:75]:
#    layer.trainable = False
# for layer in model.layers[75:95]:
#    layer.trainable = True
# for layer in model.layers[95:]:
#    layer.trainable = False
# model.load_weight(s1_weight)
# model.compile(optimizer=SGD(lr=0.005, momentum=0.9), loss='categorical_crossentropy',metrics=['categorical_accuracy'])
# his=model.fit_generator(minibatches(X_1_train,Y_1_train,batch_size=6,shuffle=False,aug=None),
#                             steps_per_epoch=len(X_1_train)//6,
#                             callbacks = [ModelCheckpoint(s1_weight, monitor='val_loss', save_best_only=True)],
#                             validation_data=minibatches(X_1_test,Y_1_test,batch_size=6,shuffle=False,aug=None),
#                             validation_steps=len(X_1_test)//6,
#                             epochs=20)

In [20]:
# for layer in model.layers[0:55]:
#    layer.trainable = False
# for layer in model.layers[55:75]:
#    layer.trainable = True
# for layer in model.layers[75:]:
#    layer.trainable = False
# model.load_weight(s1_weight)
# model.compile(optimizer=SGD(lr=0.005, momentum=0.9), loss='categorical_crossentropy',metrics=['categorical_accuracy'])
# his=model.fit_generator(minibatches(X_1_train,Y_1_train,batch_size=6,shuffle=False,aug=None),
#                             steps_per_epoch=len(X_1_train)//6,
#                             callbacks = [ModelCheckpoint(s1_weight, monitor='val_loss', save_best_only=True)],
#                             validation_data=minibatches(X_1_test,Y_1_test,batch_size=6,shuffle=False,aug=None),
#                             validation_steps=len(X_1_test)//6,
#                             epochs=20)

In [21]:
for layer in model.layers[0:35]:
   layer.trainable = False
for layer in model.layers[35:115]:
   layer.trainable = True
for layer in model.layers[55:]:
   layer.trainable = False
model.load_weights(s1_weight)
model.compile(optimizer=SGD(lr=0.005, momentum=0.9), loss='categorical_crossentropy',metrics=['categorical_accuracy'])
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
his=model.fit_generator(minibatches(X_1_train,Y_1_train,batch_size=20,shuffle=False,aug=None),
                            steps_per_epoch=len(X_1_train)//20,
                            callbacks = [ModelCheckpoint(s1_weight, monitor='val_loss', save_best_only=True, save_weight_only=True),learning_rate_reduction],
                            validation_data=minibatches(X_1_test,Y_1_test,batch_size=6,shuffle=False,aug=None),
                            validation_steps=len(X_1_test)//20,
                            epochs=20)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
18/18 [==============================] - 7s 240ms/step - loss: 3.2068 - categorical_accuracy: 0.1556 - val_loss: 3.2257 - val_categorical_accuracy: 0.1429


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
18/18 [==============================] - 3s 183ms/step - loss: 3.2214 - categorical_accuracy: 0.1722 - val_loss: 3.5023 - val_categorical_accuracy: 0.0238
Epoch 3/20
18/18 [==============================] - 3s 184ms/step - loss: 3.2206 - categorical_accuracy: 0.2167 - val_loss: 3.1862 - val_categorical_accuracy: 0.1905
Epoch 4/20
18/18 [==============================] - 3s 182ms/step - loss: 3.3130 - categorical_accuracy: 0.1333 - val_loss: 3.4025 - val_categorical_accuracy: 0.0238
Epoch 5/20
18/18 [==============================] - 3s 182ms/step - loss: 3.2457 - categorical_accuracy: 0.1139 - val_loss: 3.2438 - val_categorical_accuracy: 0.1190
Epoch 6/20
18/18 [==============================] - 3s 184ms/step - loss: 3.3282 - categorical_accuracy: 0.0722 - val_loss: 3.1260 - val_categorical_accuracy: 0.1190
Epoch 7/20
18/18 [==============================] - 3s 184ms/step - loss: 3.0099 - categorical_accuracy: 0.3472 - val_loss: 3.2881 - val_categorical_accuracy: 0.1905
Epoc

In [22]:
for layer in model.layers[35:]:
   layer.trainable = False
for layer in model.layers[0:35]:
   layer.trainable = True
model.load_weights(s1_weight)
model.compile(optimizer=SGD(lr=0.005, momentum=0.9), loss='categorical_crossentropy',metrics=['categorical_accuracy'])
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
his=model.fit_generator(minibatches(X_1_train,Y_1_train,batch_size=20,shuffle=False,aug=None),
                            steps_per_epoch=len(X_1_train)//20,
                            callbacks = [ModelCheckpoint(s1_weight, monitor='val_loss', save_best_only=True, save_weight_only=True),learning_rate_reduction],
                            validation_data=minibatches(X_1_test,Y_1_test,batch_size=6,shuffle=False,aug=None),
                            validation_steps=len(X_1_test)//20,
                            epochs=20)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
18/18 [==============================] - 11s 362ms/step - loss: 3.4386 - categorical_accuracy: 0.1944 - val_loss: 3.4923 - val_categorical_accuracy: 0.1429


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
18/18 [==============================] - 5s 299ms/step - loss: 3.4547 - categorical_accuracy: 0.2250 - val_loss: 3.5730 - val_categorical_accuracy: 0.0238
Epoch 3/20
18/18 [==============================] - 5s 303ms/step - loss: 3.4686 - categorical_accuracy: 0.1194 - val_loss: 3.4836 - val_categorical_accuracy: 0.1905
Epoch 4/20
18/18 [==============================] - 5s 302ms/step - loss: 3.4861 - categorical_accuracy: 0.0444 - val_loss: 3.5306 - val_categorical_accuracy: 0.0238
Epoch 5/20
18/18 [==============================] - 5s 305ms/step - loss: 3.4444 - categorical_accuracy: 0.1444 - val_loss: 3.3575 - val_categorical_accuracy: 0.4286
Epoch 6/20
18/18 [==============================] - 5s 298ms/step - loss: 3.4563 - categorical_accuracy: 0.2194 - val_loss: 3.2914 - val_categorical_accuracy: 0.6190
Epoch 7/20
18/18 [==============================] - 5s 299ms/step - loss: 3.4049 - categorical_accuracy: 0.3833 - val_loss: 3.3332 - val_categorical_accuracy: 0.2619
Epoc